Задание 5. Web-scraping на языке Python

Инструкция:

Шаг 1.  Изучите материалы лекционных и практических занятий по темам 4.1 и 4.2.

Шаг 4.  Опубликуйте файл расширения ipynb на платформе Odin.

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

Шаг 2. Используя доступ к данным по API, соберите с [ресурса](https://trudvsem.ru/) данные по интересующим Вас вакансиям или вакансиям Вашего региона.
Дополнительную информацию  по данному API можно найти [здесь](https://trudvsem.ru/opendata/api). Количество записей и набор признаков определяете самостоятельно. Собранные данные сохраните в DataFrame.  

In [2]:
url = 'http://opendata.trudvsem.ru/api/v1/vacancies/'
params = {'region_code': 39, 
          'hr-agency': False
          }

response = requests.get(url, params=params)
vacancies = response.json().get('results', {}).get('vacancies', [])
#vacancies

In [3]:
# Используем список для хранения информации об одной вакансии
lst_vacs = []

for vac in vacancies:
    vacancy_data = vac.get('vacancy', {})
    salary_min = vacancy_data.get('salary_min')
    
    if salary_min and salary_min >= 30000:  # Проверяем минимум зарплаты
        dct_vac = {
            'region_code': vacancy_data.get('region', {}).get('region_code'),
            'region_name': vacancy_data.get('region', {}).get('name'),
            'salary_min': salary_min,
            'job_name': vacancy_data.get('job-name')
        }
        lst_vacs.append(dct_vac)

In [4]:
# Создаем DataFrame из списка вакансий
df = pd.DataFrame(lst_vacs)
df.head(3)

,region_code,region_name,salary_min,job_name
0,3900000000000,Калининградская область,50000,офицер фельдсвязи
1,3900000000000,Калининградская область,35000,Продавец-кассир
2,3900000000000,Калининградская область,35000,Продавец-кассир


Шаг 3. С использованием библиотеки BeautifulSoup выполните web-scraping [ресурса](https://prodoctorov.ru/). Населенный пункт и специализацию врача выбрать самостоятельно.  Количество записей и набор признаков определяете самостоятельно. Собранные данные сохраните в DataFrame.

In [5]:
# Функция для получения данных о врачах
def fetch_doctors_data(url):
    response = requests.get(url)  # Выполняем GET-запрос к указанному URL
    soup = BeautifulSoup(response.content, 'html.parser')  # Парсим HTML-страницу

    doctors = []  # Список для хранения информации о врачах
    
    # Находим все карточки врачей
    doctor_cards = soup.find_all('div', class_='b-doctor-card')

    for card in doctor_cards:
        name = card.find('span', class_='b-doctor-card__name-surname').text.strip()  # ФИО
        clinics = card.find_all('div', class_='b-doctor-card__lpu-info')  # Информация о клиниках
        
        # Информация о специализации врача
        avatar_element = card.find('img', class_='b-profile-card__img')
        specialization = avatar_element['alt'].split(',')[1].strip() if avatar_element else 'Нет данных'  # Специальность

        for clinic in clinics:
            # Проверяем наличие элемента с телефоном
            phone_element = clinic.find('span', class_='b-doctor-card__lpu-phone-num')
            phone = phone_element.text.strip() if phone_element else 'Нет данных'  # Телефон

            clinic_name_element = clinic.find('div', class_='b-doctor-card__lpu-phone-title')
            clinic_name = clinic_name_element.text.strip() if clinic_name_element else 'Нет данных'  # Название клиники
            
            # Найдем рабочие дни из доступного расписания
            schedule_days = []
            for day in clinic.find_all('div', class_='b-appointment-calendar__day'):
                if 'b-appointment-calendar__day_is_disabled' not in day['class']:  # Рабочий день
                    schedule_days.append(day.find('div', class_='b-appointment-calendar__day-name').text.strip())
            
            # Заполняем данные в словаре
            doctors.append({
                'Name': name,
                'Specialization': specialization,  # Добавляем специальность
                'Clinic': clinic_name,
                'Phone': phone,
                'Working Days': ', '.join(schedule_days)
            })

    return doctors



In [6]:
# Основная часть
url = 'https://prodoctorov.ru/kaliningrad/nevrolog/'  # Начальный URL
all_doctors = []  # Для хранения данных со всех страниц
page = 1  # Номер страницы


In [7]:
while True:
    print(f"Fetching data from page {page}...")
    doctors = fetch_doctors_data(url)
    all_doctors.extend(doctors)  # Добавляем данные текущей страницы к общим

    # Переход на следующую страницу
    response = requests.get(url)  # Получаем HTML страницы снова для использования в следующем цикле
    soup = BeautifulSoup(response.content, 'html.parser')  # Парсим HTML-страницу
   
    next_page = soup.find('span', class_='b-pagination-vuetify-imitation__item b-pagination-vuetify-imitation__item_current')
    if next_page is None:
        break
    page += 1
    url = f'https://prodoctorov.ru/kaliningrad/nevrolog/?page={page}'  # Обновляем URL для следующей страницы

Fetching data from page 1...
Fetching data from page 2...
Fetching data from page 3...
Fetching data from page 4...
Fetching data from page 5...
Fetching data from page 6...
Fetching data from page 7...
Fetching data from page 8...
Fetching data from page 9...
Fetching data from page 10...
Fetching data from page 11...
Fetching data from page 12...
Fetching data from page 13...
Fetching data from page 14...
Fetching data from page 15...
Fetching data from page 16...
Fetching data from page 17...
Fetching data from page 18...
Fetching data from page 19...
Fetching data from page 20...


In [8]:
# Создаем DataFrame из собранных данных
df = pd.DataFrame(all_doctors)

# Сохраняем в CSV файл
#df.to_csv('neurologists.csv', index=False)
#print("табличка сохранениа в: neurologists.csv")
df

,Name,Specialization,Clinic,Phone,Working Days
0,Дунаева Анна Александровна,Невролог - Калининград,Запись на приём:,+7 962 262-37-48,
1,Клименкова Алевтина Анатольевна,Невролог - Калининград,Запись на приём:,+7 401 261-58-34,
2,Нарольская Светлана Борисовна,Невролог - Калининград,Запись на приём:,+7 401 220-14-32,
3,Волков Алексей Алексеевич,Невролог,Запись на приём,+7 401 279-90-48,
4,Брадулина Ирина Леонидовна,Невролог - Калининград,Запись на приём,+7 401 261-56-39,
...,...,...,...,...,...
467,Шабанова Валентина Дмитриевна,Невролог - Калининград,Нет данных,Нет данных,
468,Шабанова Валентина Дмитриевна,Невролог - Калининград,Нет данных,Нет данных,
469,Шарапова Елена Анатольевна,Невролог - Калининград,Нет данных,Нет данных,
470,Шувалова Анна Валерьевна,Невролог - Калининград,Нет данных,Нет данных,
